## Recommendation model - Two tower model

### Import TFRS

First, install and import TFRS:

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.1 MB/s eta 0:00:00


In [2]:
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Load tha Data

In [3]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/incomplete.GOVF1O_0.1.1/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [4]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [5]:
class TwoTowerModel(tfrs.Model):
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        return self.task(user_embeddings, movie_embeddings)


Define the two models and the retrieval task.

In [22]:
# Define user and movie models
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])



### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [7]:
# Define the retrieval task
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

# Create the retrieval model
model = TwoTowerModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train the model for 3 epochs
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# Get some recommendations
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 20s 702ms/step - factorized_top_k/top_1_categorical_accuracy: 1.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0019 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0439 - factorized_top_k/top_100_categorical_accuracy: 0.1012 - loss: 33073.6750 - regularization_loss: 0.0000e+00 - total_loss: 33073.6750
Epoch 2/3
25/25 [==============================] - 16s 652ms/step - factorized_top_k/top_1_categorical_accuracy: 1.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0144 - factorized_top_k/top_50_categorical_accuracy: 0.1035 - factorized_top_k/top_100_categorical_accuracy: 0.2106 - loss: 31016.1442 - regularization_loss: 0.0000e+00 - total_loss: 31016.1442
Epoch 3/3
25/25 [==============================] - 15s 599ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categor

## Recommendation model - Wide & Deep model

## Import TRFS

In [8]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [9]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Load and preprocess the data:

In [10]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

## Build vocabularies:

In [11]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

## Define the Wide & Deep model:

In [12]:
class MovieLensModel(tfrs.Model):
    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval):
        super().__init__()

        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)


## Define user and movie models for the Wide & Deep architecture:

In [27]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), embedding_dimension),
    tf.keras.layers.Dense(embedding_dimension)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), embedding_dimension),
    tf.keras.layers.Dense(embedding_dimension)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(movie_model)
    )
)

## Compile and train the model:

In [14]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

#Set up the retrieval index and get recommendations
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# Get recommendations for user 42.
_, titles = index(np.array(["42"]))
print(f"Top 5 recommendations for user 42: {titles[0, :5]}")


Epoch 1/3
25/25 [==============================] - 25s 933ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0233 - factorized_top_k/top_5_categorical_accuracy: 0.0355 - factorized_top_k/top_10_categorical_accuracy: 0.0452 - factorized_top_k/top_50_categorical_accuracy: 0.0905 - factorized_top_k/top_100_categorical_accuracy: 0.1337 - loss: 488902.8930 - regularization_loss: 0.0000e+00 - total_loss: 488902.8930
Epoch 2/3
25/25 [==============================] - 15s 596ms/step - factorized_top_k/top_1_categorical_accuracy: 8.8000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0086 - factorized_top_k/top_50_categorical_accuracy: 0.0482 - factorized_top_k/top_100_categorical_accuracy: 0.0991 - loss: 36286.1424 - regularization_loss: 0.0000e+00 - total_loss: 36286.1424
Epoch 3/3
25/25 [==============================] - 15s 590ms/step - factorized_top_k/top_1_categorical_accuracy: 7.6000e-04 - factorized_top_k/top_5_categoric

## Recommendation model - LightFM model

## Install LightFM

In [40]:
!pip install -q lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [41]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from lightfm import LightFM

## Read the Data

In [42]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])


## Build vocabularies:

In [43]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)


## Define LightFM Model

In [44]:
lightfm_model = LightFM(no_components=64, loss='warp')

## Train the LightFM Model

In [48]:
# Initialize lists to store user_ids, item_ids, and ratings_values
user_ids = []
item_ids = []
ratings_values = []

# Iterate over the dataset and extract relevant information
for interaction in ratings.as_numpy_iterator():
    user_id = user_ids_vocabulary(interaction['user_id'])
    item_id = movie_titles_vocabulary(interaction['movie_title'])

    # Check if 'rating' field is present
    if 'rating' in interaction:
        rating_value = interaction['rating']
    else:
        # Handle the case where 'rating' field is missing or undefined
        rating_value = 0  # Replace with an appropriate default value

    user_ids.append(user_id)
    item_ids.append(item_id)
    ratings_values.append(rating_value)

# Build the interactions matrix
from scipy.sparse import coo_matrix

interactions_matrix = coo_matrix((ratings_values, (user_ids, item_ids)))

# Initialize and fit LightFM model
from lightfm import LightFM

lightfm_model = LightFM(no_components=64, loss='warp')
lightfm_model.fit(interactions_matrix, epochs=10)


## Get Recommendations

In [55]:
# Example: Get top 3 recommendations for user 42
user_index = user_ids_vocabulary('42').numpy()  # Convert tensor to integer
n_items = len(movie_titles_vocabulary.get_vocabulary())

# Repeat user index for all item indices
user_indices = np.full(n_items, user_index)

# Predict scores for all items for the given user
item_indices = np.arange(n_items)
scores = lightfm_model.predict(user_indices, item_indices)

# Get indices of top 3 recommendations
top_item_indices = np.argsort(-scores)[:3]  # Indices of top 3 recommendations

# Get titles of top 3 recommendations
recommended_titles = [movie_titles_vocabulary.get_vocabulary()[idx] for idx in top_item_indices]

print(f"Top 3 recommendations for user 42: {recommended_titles}")



Top 3 recommendations for user 42: ['Stefano Quantestorie (1993)', 'Destiny Turns on the Radio (1995)', 'Dingo (1992)']
